In [ ]:
!pip install transformers datasets
! pip install -U accelerate>=0.20.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
raw_datasets = load_dataset("squad")
raw_datasets

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [ ]:
raw_datasets["train"][1]

{'id': '5733be284776f4190066117f',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'What is in front of the Notre Dame Main Building?',
 'answers': {'text': ['a copper statue of Christ'], 'answer_start': [188]}}

In [ ]:
from transformers import AutoTokenizer

In [ ]:
#model_checkpoint = "distilbert-base-cased"
model_checkpoint = "bert-base-uncased"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
context = raw_datasets["train"][1]["context"]
question= raw_datasets["train"][1]["question"]

inputs = tokenizer(question,context)
tokenizer.decode(inputs["input_ids"])

'[CLS] what is in front of the notre dame main building? [SEP] architecturally, the school has a catholic character. atop the main building\'s gold dome is a golden statue of the virgin mary. immediately in front of the main building and facing it, is a copper statue of christ with arms upraised with the legend " venite ad me omnes ". next to the main building is the basilica of the sacred heart. immediately behind the basilica is the grotto, a marian place of prayer and reflection. it is a replica of the grotto at lourdes, france where the virgin mary reputedly appeared to saint bernadette soubirous in 1858. at the end of the main drive ( and in a direct line that connects through 3 statues and the gold dome ), is a simple, modern stone statue of mary. [SEP]'

In [ ]:
inputs = tokenizer(question,context,max_length=100,truncation="only_second",stride=50,return_overflowing_tokens=True)

In [ ]:
for ids in inputs["input_ids"]:
  print(tokenizer.decode(ids))

[CLS] what is in front of the notre dame main building? [SEP] architecturally, the school has a catholic character. atop the main building's gold dome is a golden statue of the virgin mary. immediately in front of the main building and facing it, is a copper statue of christ with arms upraised with the legend " venite ad me omnes ". next to the main building is the basilica of the sacred heart. immediately behind the basilica is the grotto, a marian [SEP]
[CLS] what is in front of the notre dame main building? [SEP] it, is a copper statue of christ with arms upraised with the legend " venite ad me omnes ". next to the main building is the basilica of the sacred heart. immediately behind the basilica is the grotto, a marian place of prayer and reflection. it is a replica of the grotto at lourdes, france where the virgin mary reputedly appeared to saint bernadette soubirous in [SEP]
[CLS] what is in front of the notre dame main building? [SEP] sacred heart. immediately behind the basilic

In [ ]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'overflow_to_sample_mapping'])

In [ ]:
inputs['overflow_to_sample_mapping']

[0, 0, 0]

In [ ]:
inputs = tokenizer(raw_datasets["train"][:3]["question"],raw_datasets["train"][:3]["context"],max_length=100,truncation="only_second",stride=50,return_overflowing_tokens=True,return_offsets_mapping=True)


In [ ]:
inputs['overflow_to_sample_mapping']

[0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2]

In [ ]:
inputs = tokenizer(
    raw_datasets["train"][1]["question"],
    raw_datasets["train"][1]["context"],
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True
    )


In [ ]:
len(inputs.sequence_ids(0))

100

In [ ]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [ ]:
print("ALISHAN😎")

ALISHAN😎


In [ ]:
answer = raw_datasets["train"][1]["answers"]
answer

{'text': ['a copper statue of Christ'], 'answer_start': [188]}

In [ ]:
type(inputs.sequence_ids(0))

list

In [ ]:
sequence_ids = inputs.sequence_ids(0)

ctx_start = sequence_ids.index(1)
ctx_end=len(sequence_ids) - sequence_ids[::-1].index(1) -1

ctx_start,ctx_end

(13, 98)

In [ ]:
ans_start_char = answer['answer_start'][0]
ans_end_char=ans_start_char + len(answer['text'][0])

offset = inputs['offset_mapping'][0]
ans_start_char,ans_end_char

(188, 213)

In [ ]:
start_idx = 0
end_idx = 0

if offset[ctx_start][0]>ans_start_char or offset[ctx_end][1]<ans_end_char:
  print("target is (0,0)")
else:
  i = ctx_start
  for start_end_char in offset[ctx_start:]:
    start,end = start_end_char
    if start == ans_start_char:
      start_idx = i
    if end == ans_end_char:
      end_idx = i
      break
    i += 1

start_idx,end_idx

(52, 56)

In [ ]:
input_ids = inputs["input_ids"][0]
input_ids[start_idx:end_idx+1]

[1037, 6967, 6231, 1997, 4828]

In [ ]:
tokenizer.decode(input_ids[start_idx:end_idx+1])

'a copper statue of christ'

In [ ]:
def find_answer_token_idx(
    ctx_start,
    ctx_end,
    ans_start_char,
    ans_end_char,
    offset):
  start_idx = 0
  end_idx = 0

  if offset[ctx_start][0]>ans_start_char or offset[ctx_end][1]<ans_end_char:
    #print("target is (0,0)")
    pass
  else:
    i = ctx_start
    for start_end_char in offset[ctx_start:]:
      start,end = start_end_char
      if start == ans_start_char:
        start_idx = i
      if end == ans_end_char:
        end_idx = i
        break
      i += 1

  return start_idx,end_idx

In [ ]:
start_idxs = []
end_idxs = []

for i, offset in enumerate(inputs['offset_mapping']):
  sequence_ids = inputs.sequence_ids(i)

  ctx_start = sequence_ids.index(1)
  ctx_end = len(sequence_ids) - sequence_ids[::-1].index(1)-1

  start_idx,end_idx = find_answer_token_idx(ctx_start,ctx_end,ans_start_char,ans_end_char,offset)

  start_idxs.append(start_idx)
  end_idxs.append(end_idx)

start_idxs,end_idxs

([52, 16, 0], [56, 20, 0])

In [ ]:
for q in raw_datasets["train"]["question"][:1000]:
  if q.strip() != q:
    print(q)

In what city and state did Beyonce  grow up? 
 The album, Dangerously in Love  achieved what spot on the Billboard Top 100 chart?
Which song did Beyonce sing at the first couple's inaugural ball? 
What event did Beyoncé perform at one month after Obama's inauguration? 
Where was the album released? 
What movie influenced Beyonce towards empowerment themes? 


In [ ]:
max_length = 384
stride = 128

def tokenize_fn_train(batch):
  questions = [q.strip() for q in batch["question"]]

  inputs = tokenizer(
      questions,
      batch["context"],
      max_length = max_length,
      truncation="only_second",
      stride=stride,
      return_overflowing_tokens=True,
      return_offsets_mapping=True,
      padding="max_length"
  )

  offset_mapping = inputs.pop("offset_mapping")
  orig_sample_idxs = inputs.pop("overflow_to_sample_mapping")
  answers = batch['answers']
  start_idxs,end_idxs = [],[]

  for i,offset in enumerate(offset_mapping):
    sample_idx = orig_sample_idxs[i]
    answer = answers[sample_idx]

    ans_start_char = answer['answer_start'][0]
    ans_end_char = ans_start_char + len(answer['text'][0])

    sequence_ids = inputs.sequence_ids(i)

    ctx_start = sequence_ids.index(1)
    ctx_end = len(sequence_ids) - sequence_ids[::-1].index(1)-1
    start_idx,end_idx = find_answer_token_idx(ctx_start,ctx_end,ans_start_char,ans_end_char,offset)

    start_idxs.append(start_idx)
    end_idxs.append(end_idx)

  inputs["start_positions"] = start_idxs
  inputs["end_positions"] = end_idxs
  return inputs


In [ ]:
train_dataset = raw_datasets["train"].map(tokenize_fn_train,batched=True,remove_columns=raw_datasets["train"].column_names)

len(raw_datasets["train"]), len(train_dataset)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

(87599, 88524)

In [ ]:
raw_datasets["validation"][0]

{'id': '56be4db0acb8001400a502ec',
 'title': 'Super_Bowl_50',
 'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
 'question': 'Which NFL team represented the AFC at Super Bowl 50?',
 'answers': {'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],


In [ ]:
def tokenize_fn_validation(batch):
  questions = [q.strip() for q in batch["question"]]

  inputs = tokenizer(
      questions,
      batch["context"],
      max_length = max_length,
      truncation="only_second",
      stride=stride,
      return_overflowing_tokens=True,
      return_offsets_mapping=True,
      padding="max_length"
  )

  orig_sample_idxs = inputs.pop("overflow_to_sample_mapping")
  sample_ids = []


  for i in range(len(inputs["input_ids"])):
    sample_idx = orig_sample_idxs[i]
    sample_ids.append(batch['id'][sample_idx])

    sequence_ids = inputs.sequence_ids(i)
    offset = inputs["offset_mapping"][i]
    inputs["offset_mapping"][i] = [x if sequence_ids[j] == 1 else None for j, x in enumerate(offset)]

  inputs['sample_id'] = sample_ids
  return inputs

In [ ]:
validation_dataset = raw_datasets["validation"].map(
    tokenize_fn_validation,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)

len(raw_datasets["validation"]),len(validation_dataset)

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

(10570, 10784)

In [ ]:
from datasets import load_metric

metric = load_metric("squad")

<ipython-input-33-0eb82bc40e87>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad")


In [ ]:
predicted_answers =[
    {'id':'1','prediction_text':'Albert Einstein'},
    {'id':'2','prediction_text':'physicist'},
    {'id':'3','prediction_text':'general relativity'},
]

true_answers =[
    {'id':'1','answers':{'text':['Albert Einstein'],'answer_start':[100]}},
    {'id':'2','answers':{'text':['physicist'],'answer_start':[100]}},
    {'id':'1','answers':{'text':['special relativity'],'answer_start':[100]}},
]

metric.compute(predictions=predicted_answers,references=true_answers)

{'exact_match': 66.66666666666667, 'f1': 66.66666666666667}

In [ ]:
small_validation_dataset = raw_datasets["validation"].select(range(100))

trained_checkpoint = "distilbert-base-cased-distilled-squad"

tokenizer2 = AutoTokenizer.from_pretrained(trained_checkpoint)


old_tokenizer = tokenizer
tokenizer = tokenizer2

small_validation_processed = small_validation_dataset.map(
    tokenize_fn_validation,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names
)

tokenizer = old_tokenizer

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
import torch
from transformers import AutoModelForQuestionAnswering

small_model_inputs = small_validation_processed.remove_columns(
    ["sample_id","offset_mapping"])

small_model_inputs.set_format("torch")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

small_model_inputs_gpu = {
    k:small_model_inputs[k].to(device) for k in small_model_inputs.column_names
}

trained_model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint).to(device)

with torch.no_grad():
  outputs = trained_model(**small_model_inputs_gpu)

cuda


In [ ]:
outputs

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[ -2.2607,  -5.1783,  -5.2709,  ...,  -9.5243,  -9.5183,  -9.5288],
        [ -2.5961,  -5.5482,  -5.5313,  ...,  -9.9598,  -9.9533,  -9.9860],
        [ -3.7127,  -7.1848,  -8.5388,  ..., -11.6557, -11.6571, -11.6505],
        ...,
        [ -2.0260,  -4.4167,  -4.4980,  ...,  -8.1479,  -8.1530,  -8.1760],
        [ -4.1553,  -5.8304,  -7.1643,  ..., -10.5255, -10.5251, -10.4890],
        [ -3.2000,  -5.8162,  -6.7249,  ...,  -9.4935,  -9.5038,  -9.4871]],
       device='cuda:0'), end_logits=tensor([[ -0.7353,  -4.9236,  -5.1048,  ...,  -8.8734,  -8.8916,  -8.8550],
        [ -1.3056,  -5.3870,  -5.4945,  ...,  -9.4895,  -9.5039,  -9.4959],
        [ -2.7649,  -7.2201,  -9.0916,  ..., -11.3106, -11.3414, -11.2702],
        ...,
        [ -0.0768,  -4.8210,  -4.4374,  ...,  -8.0483,  -8.0502,  -7.9903],
        [ -2.7347,  -5.3650,  -7.2549,  ..., -10.0498, -10.0661,  -9.9886],
        [ -1.0991,  -4.2569,  -6.1267,  ...,  -8

In [ ]:
start_logits = outputs.start_logits.cpu().numpy()
end_logits = outputs.end_logits.cpu().numpy()

In [ ]:
small_validation_processed['sample_id'][:5]

['56be4db0acb8001400a502ec',
 '56be4db0acb8001400a502ed',
 '56be4db0acb8001400a502ee',
 '56be4db0acb8001400a502ef',
 '56be4db0acb8001400a502f0']

In [ ]:
len(validation_dataset['sample_id'])

10784

In [ ]:
len(set(validation_dataset['sample_id']))

10570

In [ ]:
sample_id2idxs={}

for i,id_ in enumerate(small_validation_processed['sample_id']):
  if id_ not in sample_id2idxs:
    sample_id2idxs[id_] = [i]
  else:
    print("here")
    sample_id2idxs[id_].append(i)

In [ ]:
start_logits.shape,end_logits.shape

((100, 384), (100, 384))

In [ ]:
(-start_logits[0]).argsort()

array([ 46,  57,  47,  38,  39,  58,  50,  43,  45,  54,  56,  49,  13,
        42,  40,  35,  27,  31,  48,  41,  53,  44,  37,  59,  78,  15,
         0,  52,  24,  65,  81,  70,  18,  51,  55,  26,  69,  29,  28,
        75,  61,  64,  23,  36,  32,  11, 101,  62,  66,  34,  95,  30,
        63,  21,  19,  20,  17,  14,  22,  33,  68,  87, 171,  12,  76,
        71,  73,  92, 110,  84, 151,   1,  74,   2,   6,  16,  80,  79,
       105,  98,  10,  96, 136, 169, 106, 100,  93, 165,  67, 109,   8,
        90,   3, 115,  60,   5,  97,   7, 103, 102,  86,  72, 111,  89,
       108,   4,  88,  25, 132,  77, 123, 150, 124, 153,  83, 118,  82,
        85, 107, 114, 143, 164, 137, 130, 166, 159, 131,  91,   9, 144,
       139, 160,  94, 141, 128, 112, 134, 152, 170, 154, 117, 127, 104,
       140, 157, 155, 133, 145, 119, 162, 138, 135, 156, 167, 168, 126,
       148, 163, 161, 116,  99, 120, 142, 158, 125, 146, 113, 121, 147,
       149, 129, 122, 311, 312, 304, 309, 313, 310, 300, 307, 31

In [ ]:
start_logits[0][(-start_logits[0]).argsort()]

array([10.69444   ,  9.803681  ,  4.4599767 ,  4.400482  ,  2.9437776 ,
        2.701735  ,  2.012642  ,  1.5780758 ,  0.52236927,  0.02073596,
       -0.02802782, -0.04971706, -0.3857315 , -0.6945391 , -0.7979498 ,
       -0.8678062 , -0.872207  , -1.3516879 , -1.370372  , -1.3878838 ,
       -1.5135087 , -1.7355462 , -1.8827081 , -1.8932881 , -1.9078954 ,
       -1.9304959 , -2.2607315 , -2.2983875 , -2.306936  , -2.5027428 ,
       -2.5100663 , -2.5308392 , -2.5399976 , -2.671815  , -2.7323549 ,
       -2.7710226 , -2.7713668 , -2.952134  , -3.0604637 , -3.1706042 ,
       -3.2045438 , -3.5693393 , -3.5798075 , -3.666883  , -3.725064  ,
       -3.7498558 , -3.7632174 , -3.9968169 , -4.0113277 , -4.0688004 ,
       -4.0944843 , -4.195477  , -4.2383127 , -4.3323617 , -4.352419  ,
       -4.387961  , -4.3886123 , -4.3966126 , -4.6790543 , -4.703028  ,
       -4.7757573 , -4.777813  , -4.7882166 , -4.788251  , -4.822125  ,
       -4.872537  , -4.884937  , -4.898152  , -5.0720987 , -5.10

In [ ]:
small_validation_processed['offset_mapping'][0]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 [0, 5],
 [6, 10],
 [11, 13],
 [14, 17],
 [18, 20],
 [21, 29],
 [30, 38],
 [39, 43],
 [44, 46],
 [47, 56],
 [57, 60],
 [61, 69],
 [70, 72],
 [73, 76],
 [77, 85],
 [86, 94],
 [95, 101],
 [102, 103],
 [103, 106],
 [106, 107],
 [108, 111],
 [112, 115],
 [116, 120],
 [121, 127],
 [127, 128],
 [129, 132],
 [133, 141],
 [142, 150],
 [151, 161],
 [162, 163],
 [163, 166],
 [166, 167],
 [168, 176],
 [177, 183],
 [184, 191],
 [192, 200],
 [201, 204],
 [205, 213],
 [214, 222],
 [223, 233],
 [234, 235],
 [235, 238],
 [238, 239],
 [240, 248],
 [249, 257],
 [258, 266],
 [267, 269],
 [269, 270],
 [270, 272],
 [273, 275],
 [276, 280],
 [281, 286],
 [287, 292],
 [293, 298],
 [299, 303],
 [304, 309],
 [309, 310],
 [311, 314],
 [315, 319],
 [320, 323],
 [324, 330],
 [331, 333],
 [334, 342],
 [343, 344],
 [344, 345],
 [346, 350],
 [350, 351],
 [352, 354],
 [355, 359],
 [359, 360],
 [360, 361],
 [362, 369],
 [370, 37

In [ ]:
n_largest = 20
max_answer_length = 30
predicted_answers = []


for sample in small_validation_dataset:
  sample_id = sample["id"]
  context = sample["context"]

  best_score = float('-inf')
  best_answer = None

  for idx in sample_id2idxs[sample_id]:
    start_logit = start_logits[idx]
    end_logit = end_logits[idx]

    offsets = small_validation_processed[idx]['offset_mapping']

    start_indices = (-start_logit).argsort()
    end_indices = (-end_logit).argsort()

    for start_idx in start_indices[:n_largest]:
      for end_idx in end_indices[:n_largest]:

        if offsets[start_idx] is None or offsets[end_idx] is None:
          continue

        if end_idx < start_idx:
          continue

        if end_idx - start_idx + 1 > max_answer_length:
          continue

        score = start_logit[start_idx] + end_logit[end_idx]
        if score > best_score:
          best_score = score

          first_ch = offsets[start_idx][0]
          last_ch = offsets[end_idx][1]

          best_answer = context[first_ch:last_ch]

    predicted_answers.append({'id':sample_id,'prediction_text':best_answer})



In [ ]:
small_validation_dataset['answers'][0]

{'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],
 'answer_start': [177, 177, 177]}

In [ ]:
true_answers=[
    {'id':x['id'],'answers':x['answers']} for x in small_validation_dataset
]

metric.compute(predictions=predicted_answers,references=true_answers)

{'exact_match': 83.0, 'f1': 88.25000000000004}

In [ ]:
from tqdm.autonotebook import tqdm

def compute_metrics(start_logits,end_logits,processed_dataset,orig_dataset):
  sample_id2idxs = {}

  for i,id_ in enumerate(processed_dataset['sample_id']):
    if id_ not in sample_id2idxs:
      sample_id2idxs[id_] = [i]
    else:
      sample_id2idxs[id_].append(i)

  predicted_answers = []

  for sample in tqdm(orig_dataset):
     sample_id = sample['id']
     context = sample['context']

     best_score = float('-inf')
     best_answer = None

     for idx in sample_id2idxs[sample_id]:
      start_logit = start_logits[idx]
      end_logit = end_logits[idx]

      offsets = processed_dataset[idx]['offset_mapping']

      start_indices = (-start_logit).argsort()
      end_indices = (-end_logit).argsort()

      for start_idx in start_indices[:n_largest]:
        for end_idx in end_indices[:n_largest]:
          if offsets[start_idx] is None or offsets[end_idx] is None:
            continue

          if end_idx < start_idx:
            continue

          if end_idx - start_idx + 1 > max_answer_length:
            continue

          score = start_logit[start_idx] + end_logit[end_idx]
          if score > best_score:
            best_score = score


            first_ch = offsets[start_idx][0]
            last_ch = offsets[end_idx][1]

            best_answer = context[first_ch:last_ch]
      predicted_answers.append({'id':sample_id,'prediction_text':best_answer})
  true_answers = [
      {'id':x['id'], 'answers':x['answers']} for x in orig_dataset
  ]
  return metric.compute(predictions=predicted_answers,references=true_answers)
  #return metric.compute(predictions=predicted_answers,refereces=true_answers)

In [ ]:
compute_metrics(start_logits,end_logits,small_validation_processed,small_validation_dataset)

  0%|          | 0/100 [00:00<?, ?it/s]

{'exact_match': 83.0, 'f1': 88.25000000000004}

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

In [ ]:
args = TrainingArguments(
    "finetuned-squad",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer
)

trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,2.667300
1000,1.654900
1500,1.476600
2000,1.420900
2500,1.331600
3000,1.285100
3500,1.260000
4000,1.238600
4500,1.201000
5000,1.213100


Step,Training Loss
500,2.667300
1000,1.654900
1500,1.476600
2000,1.420900
2500,1.331600
3000,1.285100
3500,1.260000
4000,1.238600
4500,1.201000
5000,1.213100


TrainOutput(global_step=33198, training_loss=0.847982983796293, metrics={'train_runtime': 7844.2281, 'train_samples_per_second': 33.856, 'train_steps_per_second': 4.232, 'total_flos': 5.204482670991974e+16, 'train_loss': 0.847982983796293, 'epoch': 3.0})

In [ ]:
trainer_output = trainer.predict(validation_dataset)

In [ ]:
type(trainer_output)

transformers.trainer_utils.PredictionOutput

In [ ]:
trainer_output

PredictionOutput(predictions=(array([[-3.5527344, -7.4921875, -8.9765625, ..., -9.40625  , -9.421875 ,
        -9.4140625],
       [-2.7558594, -7.7226562, -9.0234375, ..., -9.421875 , -9.4375   ,
        -9.4296875],
       [-4.7617188, -7.2890625, -8.6484375, ..., -9.578125 , -9.5859375,
        -9.578125 ],
       ...,
       [-2.7675781, -7.6992188, -9.0234375, ..., -9.6796875, -9.703125 ,
        -9.703125 ],
       [-3.3691406, -8.5859375, -8.4375   , ..., -9.5625   , -9.53125  ,
        -9.5390625],
       [-3.5371094, -8.71875  , -8.84375  , ..., -9.6875   , -9.703125 ,
        -9.703125 ]], dtype=float32), array([[-2.765625 , -7.8671875, -8.4609375, ..., -9.5703125, -9.5625   ,
        -9.5625   ],
       [-1.8876953, -7.921875 , -8.4140625, ..., -9.546875 , -9.5390625,
        -9.5390625],
       [-4.0546875, -8.7265625, -9.5078125, ..., -9.5      , -9.4921875,
        -9.5      ],
       ...,
       [-1.6572266, -7.875    , -9.2890625, ..., -9.4765625, -9.4609375,
        -9

In [ ]:
predictions,_,_ = trainer_output

In [ ]:
predictions

(array([[-3.5527344, -7.4921875, -8.9765625, ..., -9.40625  , -9.421875 ,
         -9.4140625],
        [-2.7558594, -7.7226562, -9.0234375, ..., -9.421875 , -9.4375   ,
         -9.4296875],
        [-4.7617188, -7.2890625, -8.6484375, ..., -9.578125 , -9.5859375,
         -9.578125 ],
        ...,
        [-2.7675781, -7.6992188, -9.0234375, ..., -9.6796875, -9.703125 ,
         -9.703125 ],
        [-3.3691406, -8.5859375, -8.4375   , ..., -9.5625   , -9.53125  ,
         -9.5390625],
        [-3.5371094, -8.71875  , -8.84375  , ..., -9.6875   , -9.703125 ,
         -9.703125 ]], dtype=float32),
 array([[-2.765625 , -7.8671875, -8.4609375, ..., -9.5703125, -9.5625   ,
         -9.5625   ],
        [-1.8876953, -7.921875 , -8.4140625, ..., -9.546875 , -9.5390625,
         -9.5390625],
        [-4.0546875, -8.7265625, -9.5078125, ..., -9.5      , -9.4921875,
         -9.5      ],
        ...,
        [-1.6572266, -7.875    , -9.2890625, ..., -9.4765625, -9.4609375,
         -9.4609375

In [ ]:
start_logits,end_logits = predictions

In [ ]:
#compute_metrics(
#    start_logits,
#    end_logits,
#    validation_dataset,
#    raw_datasets["validation"]
#)

In [ ]:
trainer.save_model('my_saved_model_new')

In [ ]:
from transformers import pipeline

In [ ]:
qa=pipeline("question-answering",model="my_saved_model_new")

In [ ]:
context = "Today I went to the store to purchage a carton of milk"
question = "What did I buy?"
qa(context=context,question=question)

{'score': 0.7836897969245911,
 'start': 38,
 'end': 54,
 'answer': 'a carton of milk'}

In [ ]:
context="""Nepal, officially the Federal Democratic Republic of Nepal, is a landlocked country in South Asia. It is mainly situated in the Himalayas, but also includes parts of the Indo-Gangetic Plain."""
question = "Where is Nepal?"
qa(context=context,question=question)

{'score': 0.9453510642051697, 'start': 87, 'end': 97, 'answer': 'South Asia'}